In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from tqdm.notebook import tqdm_notebook
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.spatial.distance import squareform, pdist
import haversine as hs
from scipy.spatial.distance import cdist
from create_dist_matrix import create_dist_matrix, create_dist_mat_2
from get_avail_act import get_avail_act
from get_best_action import get_best_action
from get_or_tools_sol import or_solution
from get_random_traj import get_random_traj
from train import train_model
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import random
from random import sample,choice
import os
from os import chdir as cd
from typing import Dict, Any, List, Tuple
import json
import requests
import folium
import polyline

In [2]:
import pandas as pd

data_csv = pd.read_csv("clean_data.csv")
df = pd.DataFrame(data_csv)


In [3]:
df.drop_duplicates(subset=['CourierId','Latitude','Longitude'],keep='first',inplace=True)

df

,CourierId,Latitude,Longitude
0,324293,55.605452,12.583987
3,324293,55.631359,12.656990
8,324293,55.638429,12.634550
12,324293,NaN,NaN
15,324293,55.656790,12.604109
...,...,...,...
254923,0,55.569114,12.618886
254924,0,55.568978,12.619223
254925,0,55.568047,12.618471
254926,0,55.570509,12.620062


In [4]:
df = df.dropna(axis=0)
index_list = [i for i in range(len(df))]
df = df.set_index([index_list])

df

,CourierId,Latitude,Longitude
0,324293,55.605452,12.583987
1,324293,55.631359,12.656990
2,324293,55.638429,12.634550
3,324293,55.656790,12.604109
4,324293,55.657004,12.604765
...,...,...,...
67658,0,55.569114,12.618886
67659,0,55.568978,12.619223
67660,0,55.568047,12.618471
67661,0,55.570509,12.620062


In [5]:
from random import random


vehicles_dict = {"Diesel Car":121.5,"Petrol Car":123.4,"LPG":83,"motorcycle":27,"Bus":822,"Truck":180.5,"Van":147,"bike":5}
vehicles_array = ["Diesel Car","Petrol Car","LPG","motorcycle","Bus","Truck","Van"]



length = len(set(df.CourierId))


bike_ids = sample(list(set(df.CourierId)),int(length*0.6))

id_vehicle_dict = {i:(lambda x:"bike" if x in bike_ids else choice(vehicles_array))(i) for i in set(df.CourierId)}

df['Vehicle'] = [id_vehicle_dict[i] for i in df.CourierId]
df['CO2 emission (g/km)'] = [vehicles_dict[j] for j in df.Vehicle]

df


,CourierId,Latitude,Longitude,Vehicle,CO2 emission (g/km)
0,324293,55.605452,12.583987,bike,5.0
1,324293,55.631359,12.656990,bike,5.0
2,324293,55.638429,12.634550,bike,5.0
3,324293,55.656790,12.604109,bike,5.0
4,324293,55.657004,12.604765,bike,5.0
...,...,...,...,...,...
67658,0,55.569114,12.618886,motorcycle,27.0
67659,0,55.568978,12.619223,motorcycle,27.0
67660,0,55.568047,12.618471,motorcycle,27.0
67661,0,55.570509,12.620062,motorcycle,27.0


In [6]:
def format_coords(coords: np.ndarray) -> str:
    coords = ";".join([f"{lon:f},{lat:f}" for lat, lon in coords])
    return coords

In [7]:
def format_options(options: Dict[str, str]) -> str:
    options = "&".join([f"{k}={v}" for k, v in options.items()])
    return options

In [8]:
class Connection:
    METERS_PER_KILOMETER = 1000.00
    SEC_PER_HOUR = 3600

    def __init__(self, host: str = None, port: str = None):
        self.host = host
        self.port = port
    def make_request(
            self,
            service: str,
            coords: np.ndarray,
            options: dict = None) -> Dict[str, Any]:
        coords = format_coords(coords)
        options = format_options(options) if options else ""
        url = f"http://{self.host}:{self.port}/{service}/v1/bike/{coords}?{options}"
        r = requests.get(url)
        req=r.json()
        #print(req)

        return req

    def trip_request(self, coords: np.ndarray, resolution='low'
                       ) ->dict[str,Any]:
        assert resolution in ('low', 'high')
        if resolution == 'low':
            options = {'roundtrip': 'true','annotations':'distance','overview':'full'}
        elif resolution == 'high':
            options = {'roundtrip': 'true','annotations':'distance','overview':'full'}
        x = self.make_request(service='trip', coords=coords, options=options)
        return x

    def route_polyline(self,input_data : dict[str,Any]) -> List[Tuple[float, float]]:
        return polyline.decode(input_data['trips'][0]['geometry'])

In [9]:
host = "34.175.118.46"
port = "80"
conn = Connection()
conn.__init__(host,port)

stops_df = df.copy()
data = list()

for CourierId, group in stops_df.groupby('CourierId'):
    group['seq_num'] = list(range(len(group)))
    data.append(group)
stops_df = pd.concat(data, axis=0)



In [10]:
id_c = 324293
stops_df.loc[df.CourierId == id_c]

,CourierId,Latitude,Longitude,Vehicle,CO2 emission (g/km),seq_num
0,324293,55.605452,12.583987,bike,5.0,0
1,324293,55.631359,12.656990,bike,5.0,1
2,324293,55.638429,12.634550,bike,5.0,2
3,324293,55.656790,12.604109,bike,5.0,3
4,324293,55.657004,12.604765,bike,5.0,4
306,324293,55.672459,12.563339,bike,5.0,5
307,324293,55.663933,12.515748,bike,5.0,6
308,324293,55.697636,12.544091,bike,5.0,7
309,324293,55.683277,12.571024,bike,5.0,8
310,324293,55.655527,12.536941,bike,5.0,9


In [11]:
df

,CourierId,Latitude,Longitude,Vehicle,CO2 emission (g/km)
0,324293,55.605452,12.583987,bike,5.0
1,324293,55.631359,12.656990,bike,5.0
2,324293,55.638429,12.634550,bike,5.0
3,324293,55.656790,12.604109,bike,5.0
4,324293,55.657004,12.604765,bike,5.0
...,...,...,...,...,...
67658,0,55.569114,12.618886,motorcycle,27.0
67659,0,55.568978,12.619223,motorcycle,27.0
67660,0,55.568047,12.618471,motorcycle,27.0
67661,0,55.570509,12.620062,motorcycle,27.0


In [12]:
"""id = 324293"""
stops_df

,CourierId,Latitude,Longitude,Vehicle,CO2 emission (g/km),seq_num
255,0,55.686158,12.546712,motorcycle,27.0,0
256,0,55.690656,12.542281,motorcycle,27.0,1
257,0,55.693217,12.547336,motorcycle,27.0,2
258,0,55.687958,12.546032,motorcycle,27.0,3
259,0,55.700277,12.551320,motorcycle,27.0,4
...,...,...,...,...,...,...
18581,325220,55.658267,12.524242,bike,5.0,264
18582,325220,55.652839,12.526648,bike,5.0,265
18583,325220,55.652494,12.526744,bike,5.0,266
18584,325220,55.659813,12.524874,bike,5.0,267


# Loop that iterates over all data set 

In [13]:

for k in reversed(list(set(df.CourierId))):
    stops_df = stops_df.loc[df.CourierId == k]
    distances = []
    # Iterate over route ID's
    for CourierId, group in stops_df.groupby('CourierId'):
        group = group.sort_values(by=['seq_num'])
        color = np.random.choice([
            "#ff0000", # red
            "#ff9500", # orange
            "#ffd900", # yellow
            "#73ff00", # green
            "#00ffd5", # teal
            "#00c3ff", # light blue
            "#0022ff", # dark blue
            "#9d00ff", # purple
            "#ff00ee", # pink
            "#ff00a2", # magenta?
        ])
        trip = conn.trip_request(
            group[['Latitude','Longitude']].values, resolution='high')

        print(trip)
        #polylines = np.array(conn.route_polyline(trip))
        distances.append(trip['trips'][0]['distance'])
        orsm_distance = trip['trips'][0]['distance']

    alpha = 0.012
    gamma = 0.4
    def tsp_solver(dist_mat,alpha=0.2,gamma=0.8):
        #alpha is the learning rate
        #gamma is the discount factor
        n_dest = dist_mat.shape[0]
        # Train RL model
        q = train_model(dist_mat, n_train = 1000, gamma = 0.2, alpha = alpha)# Get trained transition matrix

        #print(q)

        # Use model to find optimum trajectory
        state = [0]
        distance_travel = 0.
        posible_actions = get_avail_act(state, n_dest)
        while posible_actions: # until all destinations are visited
            action = get_best_action(state[-1], posible_actions, q)
            distance_travel += dist_mat[state[-1], action]
            state.append(action)
            posible_actions = get_avail_act(state, n_dest)

        #Back to warehouse
        action = 0
        distance_travel += dist_mat[state[-1], action]
        state.append(action)

        # Get Best optimization possible
        #print("\nGoogle Results: ")
        best_dist, google_route = or_solution(dist_mat)

        # Get random tour
        random_dist, random_route = get_random_traj(dist_mat)

        #Out RL results
        traj =' -> '.join([str(b) for b in state])
        #print(f"Best trajectory found with RL: \n {traj}" )
        #print(f"Total distance travelled with this traj: {distance_travel}\n")

        lat_long_array = [(df.Latitude[i],df.Longitude[i]) for i in state]

        slow_pctg = 100*(-1+distance_travel/best_dist)
        random_pctg = 100*(-1+distance_travel/random_dist)
        return slow_pctg, traj, distance_travel, google_route, best_dist, lat_long_array
        #print(f"RL solution is {100*(-1+distance_travel/best_dist)}% slower than google's solution")

    dist_list = [0] * len(df)
    #df['Distance'] = dist_list
    new_df = df.loc[df.CourierId == k]
    new_df = new_df.set_index([[i for i in range(len(new_df))]])
    dist_matrix = [[0]*len(new_df) for _ in range(len(new_df))]

    for j in new_df.index:
        for k in new_df.index:
            loc1=(new_df.Latitude[j], new_df.Longitude[j])
            loc2=(new_df.Latitude[k], new_df.Longitude[k])
            dist_matrix[j][k] = hs.haversine(loc1,loc2)

    dist_matrix = np.array(dist_matrix)
    dist_matrix = np.array([list(map(lambda x:int(np.round(1000*x)),i)) for i in dist_matrix])

    best_pctg = 100
    alpha = 0.01
    gamma = 0.2

    for _ in range(10):
            slow_pctg, rl_route, rl_dist, google_route, google_dist,coords_array = tsp_solver(dist_matrix, alpha=alpha, gamma=gamma)
            exact_pctg = 100*(-1+rl_dist/orsm_distance)
            if exact_pctg < best_pctg:
                best_pctg = exact_pctg
                if exact_pctg < 0:
                    best_coords = coords_array
                    best_dist = rl_dist

    em_fact = df.loc[df.CourierId == k]["CO2 emission (g/km)"][0]
    co2_amount_kg = best_dist*em_fact/(10**6)
    print("Courier ID :",k)
    print("CO2 Amount emitted:",co2_amount_kg,"kg")
    print("Distance traveled :",best_dist,"m")
        

ConnectTimeout: HTTPConnectionPool(host='34.175.118.46', port=80): Max retries exceeded with url: /trip/v1/bike/12.611166,55.637232;12.611669,55.637522;12.612455,55.637418;12.616479,55.641238;12.609585,55.637845;12.610353,55.639108;12.612994,55.639309;12.613044,55.639154;12.613562,55.639256;12.616381,55.639308;12.616656,55.639363;12.616543,55.640202;12.616672,55.640225;12.617032,55.639990;12.615222,55.640358;12.612682,55.640748;12.611002,55.639772;12.610065,55.639611;12.611576,55.637885;12.611900,55.640024;12.608427,55.640289;12.611157,55.637825;12.610643,55.638632;12.611206,55.637842;12.612196,55.639098;12.613534,55.639302;12.613421,55.639620;12.614040,55.639440;12.615509,55.639163;12.615847,55.639207;12.616074,55.639263;12.616978,55.640099;12.614737,55.640273;12.614094,55.640193;12.609562,55.639841;12.610511,55.639966;12.616035,55.642114;12.610233,55.641260;12.610297,55.641267;12.609989,55.642239;12.614540,55.642105;12.613232,55.641370;12.614481,55.641639;12.611910,55.641337;12.61196


Best solution so far with parameters alpha:0.01, gamma:0.2, is 15.8% FASTER than orsm's solution
Model's distance 43273.0 meters.
ORSM's distance 51375.6 meters.

Best solution so far with parameters alpha:0.01, gamma:0.2, is 21.0% FASTER than orsm's solution
Model's distance 40600.0 meters.
ORSM's distance 51375.6 meters.

Best solution so far with parameters alpha:0.01, gamma:0.2, is 24.8% FASTER than orsm's solution
Model's distance 38615.0 meters.
ORSM's distance 51375.6 meters.


[(55.605452, 12.583987),
 (55.65679, 12.604109),
 (55.657004, 12.604765),
 (55.666729, 12.623844),
 (55.666922, 12.624307),
 (55.68444, 12.606241),
 (55.684063, 12.541127),
 (55.664992, 12.622379),
 (55.65963, 12.604466),
 (55.665391, 12.626504),
 (55.666784, 12.624088),
 (55.629509, 12.639366),
 (55.660381, 12.596661),
 (55.645927, 12.642146),
 (55.637848, 12.580652),
 (55.633779, 12.650921),
 (55.638429, 12.63455),
 (55.631359, 12.65699),
 (55.605452, 12.583987)]

In [ ]:
print("="*75)
print(f"\tCO2 Consumed by Courier {id} is :",end=" ")
em_fact = df.loc[df.CourierId == id]["CO2 emission (g/km)"][0]
print(np.round(best_dist/1000 * em_fact,decimals=4),"g")
print("\t\t\t\t\t   ",np.round(best_dist/1000 * em_fact/1000,decimals=4),"kg")

	CO2 Consumed by Courier 324293 is : 193.075 g
					    0.1931 kg


In [ ]:
df.loc[df.CourierId == id]["CO2 emission (g/km)"][0]

5.0

In [14]:
count = 0
# Create map
m = folium.Map(
    location=df[['Latitude','Longitude']].mean(),
    zoom_start=12)
distances = []

trip = conn.trip_request(best_coords)

polylines = np.array(conn.route_polyline(trip))
distances.append(trip['trips'][0]['distance'])
print("Couried ID :324293", "   Distance Traveled :",trip['trips'][0]['distance'],"meters")

color = np.random.choice([
        "#ff0000", # red
        "#ff9500", # orange
        "#ffd900", # yellow
        "#73ff00", # green
        "#00ffd5", # teal
        "#00c3ff", # light blue
        "#0022ff", # dark blue
        "#9d00ff", # purple
        "#ff00ee", # pink
        "#ff00a2", # magenta?
    ])
    
for i in range(len(best_coords)-1):
    coords = best_coords[i:i+2]
    
    # Add polyline between stops
    folium.PolyLine(
        locations=polylines,
        tooltip="Courier Id 324293",
        popup=f"Distance: {trip['trips'][0]['distance']:.2f} mi., duration: {trip['trips'][0]['duration']:.1f} min.",
        color=color,
        opacity=0.35,
    ).add_to(m)

    for row in best_coords:
        lat, lon = row[0], row[1]
        if count == 0:
            print("yay")
            folium.Marker(
                location=(lat, lon),
                tooltip='Start',
                popup=folium.Popup(f"({lat:.4f}, {lon:.4f})", max_width=9999),
                icon=folium.Icon(color='green', icon='home'),
            ).add_to(m)
        elif count == (len(best_coords)-1):
            print("ayy")
            folium.Marker(
                location=(lat, lon),
                tooltip='End',
                popup=folium.Popup(f"({lat:.4f}, {lon:.4f})", max_width=9999),
                icon=folium.Icon(color='black', icon='home'),
            ).add_to(m)
        else:
            folium.CircleMarker(
                location=(lat, lon),
                tooltip=f"324293-{count}",
                radius=4,
                fill=True,
                fill_opacity=.5,
                color=color,
            ).add_to(m)
        count += 1

m.save(f"courier{id}file.html")

m

NameError: name 'best_coords' is not defined